In [1]:
import os
import torch
import transformers as ppb
os.environ['TRANSFORMERS_OFFLINE'] = '1'
    
# def load_checkpoint(checkpoint_fpath, model):
#     """
#     checkpoint_fpath: path to save checkpoint
#     model: model that we want to load checkpoint parameters into       
#     optimizer: optimizer we defined in previous training
#     """
    
#     encoder, classifier = model
    
#     # load check point
#     checkpoint = torch.load(checkpoint_fpath)

#     # initialize state_dict from checkpoint to model
#     classifier.load_state_dict(checkpoint['classifier_state_dict'])
#     encoder.load_state_dict(checkpoint['encoder_state_dict'])

#     # return model, optimizer, epoch value, min validation loss 
#     return (encoder, classifier)

In [5]:
checkpoint = torch.load('weights/checkpoints-avg-Walmart_30k.parquet/0_2_best.pt')

In [9]:
base_encoder = ppb.DistilBertModel.from_pretrained('distilbert-base-uncased')
base_encoder_state = base_encoder.state_dict()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
encoder = base_encoder
encoder.load_state_dict(checkpoint['encoder_state_dict'])

<All keys matched successfully>

In [10]:
encoder.save_pretrained("onnx/")

In [2]:
tokenizer = ppb.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [14]:
import numpy as np
from transformers import DistilBertTokenizerFast
from onnxruntime import ExecutionMode, InferenceSession, SessionOptions

#create the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Create the InferenceSession
options = SessionOptions()
options.intra_op_num_threads = 1
options.execution_mode = ExecutionMode.ORT_SEQUENTIAL
session = InferenceSession("onnx/test/test.onnx",options)

tokens = tokenizer.encode_plus("Our name are Khym Tac Tu")
tokens = {name: np.atleast_2d(value).astype(np.int64) for name,value in tokens.items()}

# Run (None means get all outputs)
output = session.run(None, tokens)

tokens


{'input_ids': array([[  101,  2256,  2171,  2024,  1047, 10536,  2213, 11937,  2278,
         10722,   102]], dtype=int64),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)}

In [15]:
output

[array([[[ 0.24262936,  0.15104881,  0.17318514, ...,  0.26403454,
           0.32051635,  0.32391867],
         [ 0.40355656,  0.04133132, -0.24977589, ...,  0.1722375 ,
           0.8100649 , -0.22530524],
         [ 0.35763988,  0.10383253, -0.15920484, ...,  0.02595149,
           0.57657367, -0.26459423],
         ...,
         [ 0.41987297,  0.17621408,  0.94034517, ...,  0.5561317 ,
           0.08265042,  0.37265146],
         [ 0.5976376 , -0.22609206,  0.32603344, ...,  0.42022657,
           0.18126191,  0.98615336],
         [ 0.6711411 ,  0.34828502, -0.02709696, ..., -0.42503494,
          -0.7768605 , -0.02205784]]], dtype=float32)]

In [ ]:
ids = data['ids'].to(device, dtype = torch.long)
mask = data['mask'].to(device, dtype = torch.long)
targets = data['labels']#.to(device, dtype = torch.long)
features = encoder(ids, mask)[0][:,0,:]
#I'm stuck here